In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%cd -q ..

In [4]:
import lcdb
import json
import lcpfn 
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# path = "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\data.json"

def readDatasetJson(path):
    f = open(path)

    dataset = json.load(f)
    # print(len(dataset))
    
    x = []
    y = []

    for v in dataset.values():
        x.append(v[0])
        y.append(v[1])
        
    x_true = th.Tensor(x)
    y_true = th.Tensor(y)
    
    return x_true, y_true


In [10]:
import os
def find_file(root_dir, target_filename):
    for root, dirs, files in os.walk(root_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)
    
    # If the loop completes without returning, the file was not found
    return None

def find_data():
    return find_file(os.getcwd(), 'data.json')

In [95]:
def get_data():
    return readDatasetJson(find_data())

x, y = readDatasetJson(find_data())
perm = th.randperm(x.size(0))
x = x[perm]
y = y[perm]

x_train = x[:x.shape[0]//2] 
x_test = x[x.shape[0]//2:]

y_train = y[:y.shape[0]//2] 
y_test = y[y.shape[0]//2:]

batch_size = x_train.shape[0]

def getBatch(training_set_x, training_set_y):
    return training_set_x, training_set_y

def getTrainBatchFunc():
    return partial(getBatch, x_train, y_train)

def getTestBatchFunc():
    return partial(getBatch, x_test, y_test)


tensor([0.7594, 0.9563, 0.4323,  ..., 0.7916, 0.3438, 0.7452])


In [50]:
# function producing batches for PFN training
def get_batch(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    #x_data, y_data = get_data()
    func = getTrainBatchFunc()
    x_data, y_data = func()
    
    x_data = x_data[:batch_size, :]
    y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    # print(x_data.shape)
    # print(y_data.shape)

    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()




    # x = np.arange(1, seq_len + 1)
    # y_target = np.empty((batch_size, seq_len), dtype=float)
    # y_noisy = np.empty((batch_size, seq_len), dtype=float)
    # x_data, y_data = get_data() # uses numpy rng
    
    # for i in range(batch_size):    
    #     if noisy_target:
    #         y_noisy[i] = y_data[i]
    #         y_target[i] = y_data[i]
    #     else:
    #         y_target[i], y_noisy[i] = y_data[i]
    # # turn numpy arrays into correctly shaped torch tensors & move them to device
    # x = (
    #     th.Tensor(x_data[0])
    #     .repeat((num_features, batch_size, 1))
    #     .transpose(2, 0)
    #     .to(device)
    # )
    # y_target = th.from_numpy(y_target).transpose(1, 0).to(device)
    # y_noisy = th.from_numpy(y_noisy).transpose(1, 0).to(device)

    # # changes
    # x = x.float()
    # y_target = y_target.float()
    # y_noisy = y_noisy.float()
    
    # return x, y_noisy, y_target

    return x_data, y_data, y_data_noisy

In [24]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=10,
                         epochs=1)

#th.save(result[2].state_dict(), "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deepdeep-learning-project\\notebooks\\model.txt")

Using cpu:0 device
init dist
Not using distributed
DataLoader.__dict__ {'num_steps': 100, 'get_batch_kwargs': {'batch_size': 10, 'eval_pos_seq_len_sampler': <function train.<locals>.eval_pos_seq_len_sampler at 0x00000253E807E550>, 'seq_len_maximum': 26, 'device': 'cpu:0', 'num_features': 1, 'hyperparameters': {}}, 'num_features': 1}
Style definition: None
Using a Transformer with 6.46 M parameters
tensor([[1.0000, 0.4175, 0.5639, 0.9750, 0.9364, 0.9925, 0.5800, 1.0000, 0.9177,
         0.1495],
        [1.0000, 0.3583, 0.5532, 0.9809, 0.9372, 0.9756, 0.5513, 1.0000, 0.9925,
         0.2750],
        [1.0000, 0.3936, 0.5537, 0.9738, 0.9368, 0.9388, 0.5587, 1.0000, 0.9913,
         0.2174],
        [1.0000, 0.3937, 0.7225, 0.9698, 0.9364, 0.9200, 0.5760, 1.0000, 0.9925,
         0.2000],
        [1.0000, 0.3936, 0.5861, 0.9788, 0.9365, 0.8663, 0.5806, 1.0000, 0.9636,
         0.2356],
        [1.0000, 0.3934, 0.6088, 0.9914, 0.9370, 0.8602, 0.6554, 1.0000, 0.9713,
         0.2625],
     

In [75]:
def get_test_batch(
    batch_size,
    num_features = 1,
    seq_len = 26,
    device = "cpu",
):
    assert num_features == 1
    print(batch_size)
    #x_data, y_data = get_data()
    func = getTestBatchFunc()
    x_data, y_data = func()
    
    # x_data = x_data[:batch_size, :]
    # y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    # print(x_data.shape)
    # print(y_data.shape)
    
    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()




    # x = np.arange(1, seq_len + 1)
    # y_target = np.empty((batch_size, seq_len), dtype=float)
    # y_noisy = np.empty((batch_size, seq_len), dtype=float)
    # x_data, y_data = get_data() # uses numpy rng
    
    # for i in range(batch_size):    
    #     if noisy_target:
    #         y_noisy[i] = y_data[i]
    #         y_target[i] = y_data[i]
    #     else:
    #         y_target[i], y_noisy[i] = y_data[i]
    # # turn numpy arrays into correctly shaped torch tensors & move them to device
    # x = (
    #     th.Tensor(x_data[0])
    #     .repeat((num_features, batch_size, 1))
    #     .transpose(2, 0)
    #     .to(device)
    # )
    # y_target = th.from_numpy(y_target).transpose(1, 0).to(device)
    # y_noisy = th.from_numpy(y_noisy).transpose(1, 0).to(device)

    # # changes
    # x = x.float()
    # y_target = y_target.float()
    # y_noisy = y_noisy.float()
    
    # return x, y_noisy, y_target

    return x_data, y_data, y_data_noisy

In [110]:
half_len = y_test.shape[1]//2
seq_len = 26

result[2].load_state_dict(th.load("C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\\model.txt"))
model = result[2]

#they get transposed somewhere idfk how or why

y_test = y_test.transpose(0, 1)
x_test = x_test.transpose(0, 1)

x_test, y_test, _ = get_test_batch(y_test.shape[0])

for x_line, y_line in zip(x_test, y_test):
    target = y_line[half_len:]
    predictions = []

    for i in range(seq_len//2, seq_len):
        predictions.append(model((x_line[:i], y_line[:i]), single_eval_pos = 0))
    print(predictions)
    #prediction = th.Tensor(predictions)

# predictions = []
# targets = y_test[:, half_len:]
# for i in range(seq_len//2, seq_len):
#     predictions.append(model(tuple(None, x_test[:i], y_test[:i])))

    

1651


ValueError: only one element tensors can be converted to Python scalars